# Body Rate from Quaternions

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, r"../../")
import AttitudeKinematicsLib as ak

In [2]:
print("Contents of AttitudeKinematicsLib:")
for name in sorted(dir(ak)):
    if not name.startswith("_"):
        print(name)

Contents of AttitudeKinematicsLib:
BInvmat_CRP
BInvmat_EP
BInvmat_Euler
BInvmat_MRP
BInvmat_PRV
Bmat_CRP
Bmat_EP
Bmat_Euler
Bmat_MRP
Bmat_PRV
CRP
CRP_to_DCM
DCM_to_CRP
DCM_to_EP
DCM_to_Euler
DCM_to_MRP
DCM_to_PRV
DCM_utils
EP_to_DCM
EulerAngles
EulerRodriguesParameters
Euler_to_DCM
MRP
MRP_to_DCM
PRV
PRV_to_DCM
integrate_quaternion
normalize_quat
np
quat_derivative
quat_diff
quat_inv
quat_mult
rotation_matrix_x
rotation_matrix_y
rotation_matrix_z
skew_symmetric
solve_ivp
validate_DCM
validate_vec3
validate_vec4


In [3]:
EPS = 1e-12

def quat_to_axis_angle(q, convention="scalar_last", shortest=True):
    """
    Convert unit quaternion to (axis, angle).
    Explicit slicing based on convention (no helper abstraction).
    """
    q = ak.normalize_quat(q)

    if convention == "scalar_last":
        v = q[:3]
        s = q[3]
    elif convention == "scalar_first":
        s = q[0]
        v = q[1:]
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    # Enforce shortest rotation (q and -q represent same attitude)
    if shortest and s < 0.0:
        v = -v
        s = -s

    v_norm = np.linalg.norm(v)
    if v_norm < EPS:
        return np.array([1.0, 0.0, 0.0]), 0.0

    angle = 2.0 * np.arctan2(v_norm, s)
    axis = v / v_norm

    return axis, angle

In [ ]:
def quat_exp_map(omega_B, t, convention="scalar_first"):
    """
    Quaternion exponential map for constant body rate:
      q_delta = exp( (t/2) * [omega_B, 0] )

    Returns a unit quaternion in the requested convention.
    """
    omega_B = np.array(omega_B, dtype=np.float64).reshape(3,)
    w = np.linalg.norm(omega_B)

    if w < EPS:
        if convention == "scalar_first":
            return np.array([1.0, 0.0, 0.0, 0.0], dtype=np.float64)
        elif convention == "scalar_last":
            return np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float64)
        else:
            raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    e_hat = omega_B / w
    phi = w * t

    c = np.cos(phi / 2.0)
    s = np.sin(phi / 2.0)

    if convention == "scalar_first":
        # [q0, q1, q2, q3] = [c, e*s]
        return np.array([c, e_hat[0]*s, e_hat[1]*s, e_hat[2]*s], dtype=np.float64)
    elif convention == "scalar_last":
        # [q1, q2, q3, q0] = [e*s, c]
        return np.array([e_hat[0]*s, e_hat[1]*s, e_hat[2]*s, c], dtype=np.float64)
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

In [4]:
def body_rate_from_q1_q2(q1, q2, t, convention="scalar_last", shortest=True):
    """
    Compute constant body-frame angular velocity ω^B that slews q1 -> q2 in time t.

    Uses library convention:
      quat_mult(a, b) = b ⊗ a
    """
    if t <= 0:
        raise ValueError("t must be > 0.")

    q1 = ak.normalize_quat(q1)
    q2 = ak.normalize_quat(q2)

    q1_inv = ak.quat_inv(q1, convention=convention)

    # Relative quaternion: q_delta = q2 ⊗ q1^{-1}
    q_delta = ak.quat_mult(q1_inv, q2, convention=convention)

    axis, angle = quat_to_axis_angle(
        q_delta,
        convention=convention,
        shortest=shortest
    )

    omega_B = (angle / t) * axis
    return omega_B, q_delta, axis, angle


In [5]:
def rk4_step_quat(q, omega_B, dt, convention="scalar_last"):
    k1 = ak.quat_derivative(q, omega_B, convention=convention)
    k2 = ak.quat_derivative(q + 0.5*dt*k1, omega_B, convention=convention)
    k3 = ak.quat_derivative(q + 0.5*dt*k2, omega_B, convention=convention)
    k4 = ak.quat_derivative(q + dt*k3, omega_B, convention=convention)

    q_next = q + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    return ak.normalize_quat(q_next)

def simulate_constant_body_rate(q0, omega_B, t, dt=0.01, convention="scalar_last"):
    q = ak.normalize_quat(q0)
    n = int(np.ceil(t / dt))
    dt_eff = t / n

    for _ in range(n):
        q = rk4_step_quat(q, omega_B, dt_eff, convention=convention)

    return q

def quat_error_sign_invariant(q_hat, q_true):
    q_hat = ak.normalize_quat(q_hat)
    q_true = ak.normalize_quat(q_true)
    return min(
        np.linalg.norm(q_hat - q_true),
        np.linalg.norm(q_hat + q_true)
    )


In [6]:
t = 10.0
theta = np.deg2rad(45.0)

# JPL/Shuster scalar-last
q1 = np.array([0.0, 0.0, 0.0, 1.0])
q2 = np.array([0.0, 0.0, np.sin(theta/2.0), np.cos(theta/2.0)])

omega_expected = np.array([0.0, 0.0, theta / t])

omega_B, q_delta, axis, angle = body_rate_from_q1_q2(
    q1, q2, t,
    convention="scalar_last",
    shortest=True
)

print("q_delta:", q_delta)
print("axis:", axis)
print("angle [deg]:", np.rad2deg(angle))
print("omega_expected [rad/s]:", omega_expected)
print("omega_B        [rad/s]:", omega_B)
print("omega error norm:", np.linalg.norm(omega_B - omega_expected))

q2_hat = simulate_constant_body_rate(
    q1, omega_B, t,
    dt=0.01,
    convention="scalar_last"
)

err = quat_error_sign_invariant(q2_hat, q2)

print("\nq2_hat:", q2_hat)
print("q2    :", ak.normalize_quat(q2))
print("sign-invariant quaternion error:", err)


q_delta: [0.         0.         0.38268343 0.92387953]
axis: [0. 0. 1.]
angle [deg]: 45.00000000000001
omega_expected [rad/s]: [0.         0.         0.07853982]
omega_B        [rad/s]: [0.         0.         0.07853982]
omega error norm: 1.3877787807814457e-17

q2_hat: [0.         0.         0.38268343 0.92387953]
q2    : [0.         0.         0.38268343 0.92387953]
sign-invariant quaternion error: 1.4946834900704542e-15


## Exp Map

In [7]:
EPS = 1e-12

def quat_exp_map(omega_B, t, convention="scalar_last"):
    """
    Closed-form quaternion exponential map for constant body rate:
      q_delta = exp( (t/2) * [omega_B, 0] )
    """
    omega_B = np.array(omega_B, dtype=np.float64).reshape(3,)
    w = np.linalg.norm(omega_B)

    if w < EPS:
        # zero rotation
        if convention == "scalar_last":
            return np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float64)
        elif convention == "scalar_first":
            return np.array([1.0, 0.0, 0.0, 0.0], dtype=np.float64)
        else:
            raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    e_hat = omega_B / w
    phi = w * t

    c = np.cos(phi / 2.0)
    s = np.sin(phi / 2.0)

    if convention == "scalar_last":
        # [x, y, z, w]
        return np.array([e_hat[0]*s, e_hat[1]*s, e_hat[2]*s, c], dtype=np.float64)
    elif convention == "scalar_first":
        # [w, x, y, z]
        return np.array([c, e_hat[0]*s, e_hat[1]*s, e_hat[2]*s], dtype=np.float64)
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")


In [8]:
q_delta_exp = quat_exp_map(omega_B, t, convention="scalar_last")

# reconstruct q2_hat_exp = q_delta_exp ⊗ q1
q2_hat_exp = ak.quat_mult(q1, q_delta_exp, convention="scalar_last")

err_qdelta = quat_error_sign_invariant(q_delta_exp, q_delta)
err_expmap = quat_error_sign_invariant(q2_hat_exp, q2)

print("q_delta (from q1,q2):", q_delta)
print("q_delta (exp-map)  :", q_delta_exp)
print("q_delta error (sign-invariant):", err_qdelta)

print("\nq2_hat (exp-map):", q2_hat_exp)
print("q2             :", ak.normalize_quat(q2))
print("sign-invariant quaternion error (exp-map):", err_expmap)


q_delta (from q1,q2): [0.         0.         0.38268343 0.92387953]
q_delta (exp-map)  : [0.         0.         0.38268343 0.92387953]
q_delta error (sign-invariant): 5.551115123125783e-17

q2_hat (exp-map): [0.         0.         0.38268343 0.92387953]
q2             : [0.         0.         0.38268343 0.92387953]
sign-invariant quaternion error (exp-map): 5.551115123125783e-17
